In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None
import numpy as np

import os

In [2]:
this_dir = os.getcwd()
os.chdir('..')
test_perform_full = pd.read_csv( 'data/test_perform_full.csv' )
os.chdir(this_dir)

In [3]:
def pretty_round_df(df, length):
    for cdx in range(len(df.columns)):
        ssplit = df.iloc[:, cdx].astype(str).str.split('.')
        if len(ssplit[0]) == 1:
            continue
        df.iloc[:, cdx] = [e[0] + '.' + e[1][:2] if len(e[1])>length
                   else e[0] + '.' + e[1].ljust(length, '0') for e in ssplit]

    return df

def get_formatted_index(df):
    sv_d = {
        'SubjectiveLit': 'Literacy',
        'TrustPhys': 'Trust',
        'wer': 'Depression'
    }
    mod_d = {
        'roberta': 'RoBERTa'
    }
    
    new_index = []
    for t in df.index:
        new_t = list(t)
        if t[0] in sv_d:
            new_t[0] = sv_d[t[0]]
        if t[1] in mod_d:
            new_t[1] = mod_d[t[1]]
        else:
            new_t[1] = t[1].upper()
        
        new_index.append(tuple(new_t))
    return pd.MultiIndex.from_tuples( new_index )



In [5]:
perform_cols = ['test_auc', 'test_acc', 'test_f1', 'irt_ability']

# order model_type
test_perform_full['model_type_ord'] = pd.Categorical( test_perform_full['model_type'], ['ffn', 'cnn', 'lstm', 'bert', 'roberta'] )

res_df = test_perform_full.groupby(['score_var', 'model_type_ord'])[ perform_cols ].max()


for c in perform_cols:
    if 'irt' not in c:
        res_df[c] = 100*res_df[c]
        
res_df = res_df.rename(columns={
    'test_auc': 'AUC', 'test_acc': 'Acc.',
    'test_f1': 'F1 Score', 'irt_ability': 'IRT Ability'})

res_df.index = get_formatted_index(res_df)


pretty_round_df( res_df, 2 ).to_latex( 'results_by_model_type_v2.tex' )

/var/folders/7y/prwzx0bj5_348vgrl_yv9c9w0000gn/T/ipykernel_52032/2921095122.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  res_df = test_perform_full.groupby(['score_var', 'model_type_ord'])[ perform_cols ].max()
/var/folders/7y/prwzx0bj5_348vgrl_yv9c9w0000gn/T/ipykernel_52032/2812878845.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if len(ssplit[0]) == 1:
/var/folders/7y/prwzx0bj5_348vgrl_yv9c9w0000gn/T/ipykernel_52032/2812878845.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['60.27', '63.02', '65.70', '70.71', '71.18', '67.